Ответ на вопрос [об инвертировании матрицы методом Гаусса](https://ru.stackoverflow.com/questions/1279499)

In [1]:
import numpy as np

In [2]:
def invert(A):
    assert np.ndim(A) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    matrix = np.hstack((A, np.eye(n)))
        
    # Приводим матрицу А к диагональному виду
    for nrow, row in enumerate(matrix):
        # nrow равен номеру строки
        # row содержит саму строку матрицы
        divider = row[nrow] # диагональный элемент
        # делим на диагональный элемент.
        row /= divider
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
            
    # обратный ход
    for nrow in range(len(matrix)-1,0,-1):
        row = matrix[nrow]
        for upper_row in matrix[:nrow]:
            factor = upper_row[nrow]
            # Вычитание целой строки на 15% быстрее, чем вычитание только правой части
            upper_row -= factor*row
    return matrix[:, n:].copy()

In [3]:
A = np.array([[3.8, 6.7, -1.2], 
              [6.4, 1.3, -2.7], 
              [2.4, -4.5, 3.5]])

In [4]:
I = invert(A)
I

array([[ 0.04128819,  0.09805945,  0.08980182],
       [ 0.15689513, -0.08790039, -0.01401625],
       [ 0.1734104 , -0.18025555,  0.206115  ]])

Проверка - произведение обратной матрицы и исходной матрицы

In [5]:
np.matmul(A, I)

array([[ 1.00000000e+00, -4.13116453e-17, -1.30511158e-17],
       [ 4.13606208e-17,  1.00000000e+00,  1.35471190e-16],
       [ 2.77555756e-17, -1.38777878e-17,  1.00000000e+00]])

Максимальное отклонение от нуля в произведении обратной и исходной матриц

In [6]:
np.max(np.abs(np.matmul(A, I) - np.eye(3)))

1.3547119009181044e-16

Исправленный вариант топик-стартера

In [7]:
def inverse_matrix(matrix_origin):
    """
    Функция получает на вход матрицу, затем добавляет к ней единичную матрицу, 
    проводит элементарные преобразования по строкам с первоначальной, добиваясь получения слева единичной матрицы.
    В этом случае справа окажется матрица, которая является обратной к заданнй первоначально 
    """
    # Склеиваем 2 матрицы: слева - первоначальная, справа - единичная
    n = matrix_origin.shape[0]
    m = np.hstack((matrix_origin, np.eye(n)))
    
    for nrow, row in enumerate(m):
        # nrow равен номеру строки
        # row содержит саму строку матрицы
        divider = row[nrow] # диагональный элемент
        # делим на диагональный элемент:
        row /= divider
        # теперь вычитаем приведённую строку из всех нижележащих строк:
        for lower_row in m[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
    # обратный ход:
    for k in range(n - 1, 0, -1):
        for row_ in range(k - 1, -1, -1):
            if m[row_, k]:
                # 1) Все элементы выше главной диагонали делаем равными нулю
                m[row_, :] -= m[k, :] * m[row_, k]
    return m[:,n:].copy()
    
            


In [8]:
matrix = np.array([[3.8, 6.7, -1.2], 
                   [6.4, 1.3, -2.7], 
                   [2.4, -4.5, 3.5]])

inverse_matrix(np.copy(matrix))

array([[ 0.04128819,  0.09805945,  0.08980182],
       [ 0.15689513, -0.08790039, -0.01401625],
       [ 0.1734104 , -0.18025555,  0.206115  ]])

In [13]:
B = np.random.rand(100,100)
%timeit -n 10 -r 3 invert(B)

21.9 ms ± 724 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [14]:
B = np.random.rand(100,100)
%timeit -n 10 -r 3 inverse_matrix(B)

25.6 ms ± 819 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
